In [252]:
%load_ext autoreload
%autoreload 2
from func.utils import get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, diff_of_days, diff_of_times
from glob import glob
from matplotlib import pyplot as plt
from pathlib import Path
from tqdm import tqdm
from train_prophet import main_prophet
import datetime
import math
import os
import numpy as np
import pandas as pd
import seaborn as sns
import sys
%matplotlib inline
sns.set()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [383]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET]

train_paths = glob('../feature/raw_main/*_train.gz')
test_paths = glob('../feature/raw_main/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count('DT') 
               or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
              ]
test_paths  = [path for path in test_paths  
               if path.count('DT') 
               or path.count('day_no') 
               or path.count('Amt')
               or path.count('Fraud') 
               or path.count('D')
               or path.count('card')
               or path.count('addr')
               or path.count('Reg')
               or path.count('P_email')
              ]

train_df = parallel_load_data(train_paths)
test_df = parallel_load_data(test_paths)

Process ForkPoolWorker-618:
Process ForkPoolWorker-617:
Traceback (most recent call last):
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/yryrgogo/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/home/yryrgogo/anaconda3/lib/python3

In [428]:
START_DATE = '2017-11-04'
# START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, '%Y-%m-%d')
train_df['datetime'] = train_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
test_df['datetime'] = test_df['TransactionDT'].apply(lambda x: (startdate + datetime.timedelta(seconds = x) - datetime.timedelta(seconds = 14400) ))
train_df['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
test_df['datetime'].fillna(datetime.date(2020, 1, 1), inplace=True)
train_df['date'] = train_df['datetime'].map(lambda x: x.date())
test_df['date']  =  test_df['datetime'].map(lambda x: x.date())

In [429]:
list_regist = []
for d, diff in tqdm(train_df[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

train_df['Regist_date'] = list_regist
    
list_regist = []
for d, diff in tqdm(test_df[['date', 'D1']].values):
    if diff < 999999:
        regist = date_add_days(d, -1*diff)
    else:
        regist = date_add_days(d, 0)
    list_regist.append(str(regist))

test_df['Regist_date'] = list_regist

100%|██████████| 506691/506691 [00:02<00:00, 206815.23it/s]


In [430]:
data = pd.concat([train_df, test_df], axis=0, ignore_index=True)

In [493]:
user_keys = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'Regist_date', 'P_emaildomain']
user_keys_nomail = ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'addr1', 'addr2', 'Regist_date']
for col in user_keys:
    data[col].fillna('##', inplace=True)
    
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', COLUMN_DT]
data[user_keys] =  data[user_keys].astype('str')

user_df = data.groupby(user_keys)[COLUMN_ID].min()
user_df = user_df.to_frame('user_id')

user_df_nomail = data.groupby(user_keys_nomail)[COLUMN_ID].min()
user_df_nomail = user_df_nomail.to_frame('user_id_nomail')
print(user_df.shape)
print(user_df_nomail.shape)

(534789, 1)
(432283, 1)


In [494]:
user_df = data.set_index(user_keys).join(user_df).join(user_df_nomail)
user_df.head()

D1  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain         
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com       0.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      83.0   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com   0.0   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com       0.0   
                                          428.0 87.0  2018-11-05  gmail.com       0.0   

                                                                                  D10  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain         
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com       0.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      83.0   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com   0.0   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com       0.0   
                                          428.0 87.0  2018-11-05  gmail.com       0.0   

                                                                                 D11  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain        
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com      NaN   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      NaN   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com  0.0   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com      NaN   
                                          428.0 87.0  2018-11-05  gmail.com      0.0   

                                                                                 D12  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain        
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com      5.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      NaN   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com  NaN   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com      0.0   
                                          428.0 87.0  2018-11-05  gmail.com      NaN   

                                                                                 D13  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain        
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com      5.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      NaN   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com  NaN   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com      0.0   
                                          428.0 87.0  2018-11-05  gmail.com      NaN   

                                                                                 D14  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain        
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com      0.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      NaN   
10001 555.0 150.0 visa       226.0 credit 204.0 87.0  2018-07-29  anonymous.com  NaN   
10002 302.0 150.0 visa       226.0 debit  ##    ##    2018-09-17  gmail.com      0.0   
                                          428.0 87.0  2018-11-05  gmail.com      NaN   

                                                                                  D15  \
card1 card2 card3 card4      card5 card6  addr1 addr2 Regist_date P_emaildomain         
1000  555.0 185.0 mastercard 224.0 debit  ##    ##    2018-01-09  gmail.com       5.0   
10000 111.0 150.0 mastercard 117.0 debit  184.0 87.0  2017-09-28  gmail.com      83.0   
10001 555.0

In [495]:
check_Ds = ['D1', 'D2', 'D3', 'D4', 'D5', 'datetime']
uid = 'user_id'
uid_2 = 'user_id_nomail'
user_df.sort_values(by=[uid, uid_2, 'datetime'], inplace=True)

In [521]:
uid_p = 'user_id_p'
uid_2_p = 'user_id_nomail_p'

sp1 = user_df[~user_df['D1'].isnull()]
sp1[uid_p] = sp1[[uid, 'ProductCD']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)
check_cols = [COLUMN_ID, uid, uid_p, COLUMN_TARGET, 'Regist_date', 'P_emaildomain'] + check_Ds
# sp1[uid_2_p] = sp1[[uid_2, 'ProductCD']].apply(lambda x: str(x[0]) + '-' + str(x[1]), axis=1)
# check_cols = [COLUMN_ID, uid, uid_2, uid_p, uid_2_p, COLUMN_TARGET, 'Regist_date', 'P_emaildomain'] + check_Ds
sp1 = sp1.reset_index()[check_cols]
sp1.sort_values(by=[uid_p, 'datetime'], inplace=True)
# sp1.sort_values(by=[uid_p, uid_2_p, 'datetime'], inplace=True)

In [525]:
def create_new_user_id(col_user_base, col_user_key):
    
    if col_user_key not in sp1.columns:
        sp1.reset_index(inplace=True)
    
    col_same_user = f'same_{col_user_key}'
    max_fraud = sp1.groupby(col_user_key)[COLUMN_TARGET].max()
    if col_user_key in sp1.columns:
        sp1.set_index(col_user_key, inplace=True)
    sp1['max_Fraud'] = max_fraud
    
    # Fraudを持つ可能性のあるユーザーのみで考える
    sp1_fraud = sp1[sp1['max_Fraud']==1]
    sp1_fraud.reset_index(inplace=True)
    diff_fraud_users = sp1_fraud[sp1_fraud[COLUMN_TARGET] != sp1_fraud['max_Fraud']].reset_index()[col_user_base].unique()
    sp1_fraud = sp1_fraud[sp1_fraud[col_user_base].isin(diff_fraud_users)]
    sp1_fraud.sort_values(by=[col_user_base, 'datetime'], inplace=True)
    
    sp1_fraud['date'] = sp1_fraud['datetime'].map(lambda x: x.date())
    sp1_fraud['diff_from_regist'] = sp1_fraud['datetime'] - sp1_fraud['Regist_date'].map(lambda x: datetime.datetime(int(x[:4]), int(x[5:7]), int(x[8:10]) ) )
    sp1_fraud['diff_from_regist'] = sp1_fraud['diff_from_regist'].map(lambda x: x.days)
    
    sp1_fraud['Regist_date_add_D1'] = sp1_fraud[['Regist_date', 'D1']].apply(lambda x: date_add_days(x[0], x[1]), axis=1)
    sp1_fraud['Regist_date_add_D1-D3'] = sp1_fraud[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]), axis=1)
    sp1_fraud['Regist_date_add_D1-D3-1'] = sp1_fraud[['Regist_date_add_D1', 'D3']].apply(lambda x: date_add_days(x[0], -1*x[1]-1), axis=1)
    
    sp1_fraud['date__before1'] = sp1_fraud.groupby(col_user_key)['date'].shift(1)
    sp1_fraud['date__before2'] = sp1_fraud.groupby(col_user_key)['date'].shift(2)
    sp1_fraud['date__before3'] = sp1_fraud.groupby(col_user_key)['date'].shift(3)
    sp1_fraud['D1__before1']   = sp1_fraud.groupby(col_user_key)['D1'].shift(1)
    sp1_fraud['D1__before2']   = sp1_fraud.groupby(col_user_key)['D1'].shift(2)
    
    sp1_fraud['diff_D1__before1'] = sp1_fraud[['D1', 'D1__before1']].apply(lambda x: x[0] - x[1] , axis=1)
    sp1_fraud['diff_D1__before2'] = sp1_fraud[['D1', 'D1__before2']].apply(lambda x: x[0] - x[1] , axis=1)
    sp1_fraud['diff_day__before1'] = sp1_fraud[['date', 'date__before1']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    sp1_fraud['diff_day__before2'] = sp1_fraud[['date', 'date__before2']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    sp1_fraud['diff_day__before3'] = sp1_fraud[['date', 'date__before3']].apply(lambda x: diff_of_days(x[0], x[1]) , axis=1)
    
    # ユーザーIDの作成
    cnt = 0
    p_cnt = 0
    list_user = []
    for user, d3, b1, b2, b3, D1_b1, D1_b2 in sp1_fraud[[col_user_key, 'D3', 'diff_day__before1', 'diff_day__before2', 'diff_day__before3', 'diff_D1__before1', 'diff_D1__before2']].values:
        p_cnt+=1
#         if d3==b3 or d3==b3-1:
        if d3==b3:
            current_user = b3_user
#         elif d3==b2 or d3==b2-1:
        elif d3==b2:
            current_user = b2_user
#         elif d3==b1 or d3==b1-1:
        elif d3==b1:
            current_user = b1_user
#         elif D1_b1 == b1 and D1_b1 != 0:
#             current_user = b1_user
#         elif D1_b2 == b2 and D1_b2 != 0:
#             current_user = b2_user
        else:
            cnt += 1
        current_user = cnt
            
        if p_cnt>=3:
            b3_user = b2_user
        if p_cnt>=2:
            b2_user = b1_user
        b1_user = cnt
        list_user.append(current_user)
        
    sp1_fraud[col_same_user] = list_user
    
    return sp1_fraud

In [526]:
col_same_user = f'same_{uid_p}'
df_new_user = create_new_user_id(uid, uid_p)

In [532]:
trans_new_user = df_new_user[COLUMN_ID].values
filepath = '../output/0826_transaction_new_user.csv'
with open(filepath, 'w') as f:
    for trans in [COLUMN_ID] + trans_new_user:
        f.write(str(trans) + '\n')

In [ ]:
df_new_user[[COLUMN_ID, col_same_user, uid_p, 'P_emaildomain', 'isFraud', 'Regist_date', 'datetime', 'D1', 'D3', 'diff_day__before1', 'diff_day__before2', 'diff_day__before3', 'diff_D1__before1', 'diff_D1__before2']]